<a href="https://colab.research.google.com/github/CastHash532/kaggle-automl/blob/main/Kaggle_news_NLP_fast_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install fast-bert

     |████████████████████████████████| 99 kB 2.6 MB/s 
     |████████████████████████████████| 3.0 MB 8.0 MB/s 
     |████████████████████████████████| 124 kB 49.9 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 769 kB 32.9 MB/s 
     |████████████████████████████████| 1.2 MB 33.1 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=654b9a82b616ede26e96714d39a836df8bb6c5cb712d96b0c955b4a0ccd19012
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


## Authenticating with Kaggle using kaggle.json

Navigate to https://www.kaggle.com. Then go to the [Account tab of your user profile](https://www.kaggle.com/me/account) and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


## Load data and preprocess



In [3]:
!kaggle datasets download -d rmisra/news-category-dataset

 35% 9.00M/25.4M [00:00<00:00, 17.5MB/s]
100% 25.4M/25.4M [00:00<00:00, 43.3MB/s]


In [4]:
!unzip -u news-category-dataset.zip

Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v2.json  


In [5]:
import pandas as pd
import numpy as np

dataset = pd.read_json('/content/News_Category_Dataset_v2.json',lines=True)

In [6]:
# WORLDPOST and THE WORLDPOST were given as two separate categories in the dataset. Here I change the category THE WORLDPOST to WORLDPOST 
dataset.category = dataset.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)
dataset.category = dataset.category.map(lambda x: "PARENTING" if x == "PARENTS" else x)

In [62]:
# Here I concatinate both headlines and short descriptions into one Series 

l=pd.DataFrame(['. ']*dataset.shape[0])[0] # seperators dataframe
dataset['text']= dataset['headline'] + l + dataset['short_description']

In [8]:
from sklearn.model_selection import train_test_split

ds_train, ds_test = train_test_split(dataset,test_size=0.2, random_state=42)

ds_train= ds_train[:8000]
ds_test= ds_test[:2000]

In [60]:
text= 'text'
target='category'

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np



encoder = OneHotEncoder()
targets = encoder.fit_transform(df[[target]])
df1= pd.DataFrame(df[text].reset_index(drop=True))
df2= pd.DataFrame(targets.toarray(), columns=encoder.categories_[0].tolist())
train =pd.merge(df1, df2, left_index=True, right_index=True)


targets = encoder.fit_transform(df[[target]])
df1= pd.DataFrame(df[text].reset_index(drop=True))
df2= pd.DataFrame(targets.toarray(), columns=encoder.categories_[0].tolist())
train =pd.merge(df1, df2, left_index=True, right_index=True)


In [ ]:
# Take a subset of the News dataset and save it to csv files
train.to_csv('/content/train.csv')
test.to_csv('/content/test.csv')
pd.DataFrame(encoder.categories_[0].tolist()).to_csv('/content/labels.csv', index=False)

In [63]:
labels = encoder.categories_[0].tolist()

## Defining Model

In [72]:
from fast_bert.data_cls import BertDataBunch
import torch

DATA_PATH = LABEL_PATH = OUTPUT_PATH ='/content'

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='test.csv',
                          label_file='labels.csv',
                          text_col=text,
                          label_col=labels,
                          batch_size_per_gpu=64,
                          max_seq_length=512,
                          multi_gpu=True if torch.cuda.device_count() > 1 else False,
                          multi_label=True,
                          model_type='bert')

In [ ]:

from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging


logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_PATH,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=True if torch.cuda.device_count() > 1 else False,
						is_fp16=True,
						multi_label=True,
						logging_steps=50,
						freeze_transformer_layers=True
						)


In [ ]:
learner.lr_find(
    start_lr=1e-5,
    num_iter=10,
    end_lr=10,
    optimizer_type='lamb')

# Fine Tuning and Validation

In [ ]:
#fine tuning the model
learner.fit(epochs=3,
			lr=5e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_cosine",
			optimizer_type="lamb")


In [ ]:
learner.validate()